# Scratch

# Structure-based VS

In [ ]:
!pip3 install mdtraj

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.9 MB/s eta 0:00:0000:0100:01


In [ ]:
import mdtraj
from typing import Tuple, List, Optional

def load_trajectory(file_path: str) -> mdtraj.Trajectory:
    """
    Load a molecular trajectory from a file.

    Args:
        file_path (str): Path to the trajectory file.

    Returns:
        mdtraj.Trajectory: Loaded trajectory object.
    """
    return mdtraj.load(file_path)

def get_protein_ligand_idxs(traj: mdtraj.Trajectory, ligand_selection: Optional[str] = None) -> Tuple[np.ndarray, np.ndarray]:
    """
    Get atom indices for protein and ligand in the trajectory.

    Args:
        traj (mdtraj.Trajectory): Input trajectory.
        ligand_selection (str, optional): MDTraj selection string for ligand. Defaults to "not protein".

    Returns:
        Tuple[np.ndarray, np.ndarray]: Atom indices for protein and ligand.
    """
    protein = traj.top.select("protein")
    ligand_selection = "not protein" if ligand_selection is None else ligand_selection
    ligand = traj.top.select(ligand_selection)
    return protein, ligand

def save_trimmed_pdb(path: str, traj: mdtraj.Trajectory, idxs: np.ndarray) -> None:
    """
    Save a new PDB file with only the specified atoms.

    Args:
        path (str): Output file path.
        traj (mdtraj.Trajectory): Input trajectory.
        idxs (np.ndarray): Atom indices to include in the output.
    """
    traj.atom_slice(idxs).save_pdb(path)

# Load the trajectory
traj = load_trajectory("data/CH08_6vhn.pdb")

# Get protein and ligand indices
receptor, ligand = get_protein_ligand_idxs(traj, "not protein and not water")

# Save ligand and receptor PDB files
save_trimmed_pdb("data/CH08_ligand.pdb", traj, ligand)
save_trimmed_pdb("data/CH08_receptor.pdb", traj, receptor)

In [ ]:
import py3Dmol
# First we assign the py3Dmol.view as view
view=py3Dmol.view()
# The following lines are used to add the addModel class
# to read the PDB files of chain B and C
view.addModel(open('data/CH08_6vhn_prepared.pdb', 'r').read(),'pdb')
view.addModel(open('data/CH08_6vhn.pdb', 'r').read(),'pdb')
# Zooming into all visualized structures
view.zoomTo()
# Here we set the background color as white and set the cartoon style
view.setBackgroundColor('white')
view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
# And we finally visualize the structures using the command below
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
!pip3 install openbabel-wheel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 2.3 MB/s eta 0:00:0000:0100:01


In [ ]:
import os
from typing import Union, List
from dataclasses import dataclass
import numpy as np
import mdtraj
import datamol as dm
import py3Dmol

try:
    from openbabel import pybel
except ImportError:
    raise ImportError("OpenBabel and Pybel are required for this script.")

@dataclass
class MoleculePreparationConfig:
    add_hydrogens: bool = True
    make_3d: bool = True
    charge_model: str = "gasteiger"

@dataclass
class Point:
    x: float
    y: float
    z: float

    @classmethod
    def from_array(cls, arr: np.ndarray):
        return cls(arr[0], arr[1], arr[2])

    def to_array(self):
        return np.array([self.x, self.y, self.z])

@dataclass
class Box:
    center: Point
    size: Point

    @classmethod
    def from_array(cls, center: np.ndarray, size: np.ndarray):
        return cls(Point.from_array(center), Point.from_array(size))

class MoleculeReader:
    @staticmethod
    def read_molecule(file: Union[str, os.PathLike], file_format: str = "pdb") -> List[pybel.Molecule]:
        return list(pybel.readfile(format=file_format, filename=str(file)))

class MoleculePreparator:
    def __init__(self, config: MoleculePreparationConfig = MoleculePreparationConfig()):
        self.config = config

    def prepare_molecule(self, molecule: pybel.Molecule) -> pybel.Molecule:
        if self.config.add_hydrogens:
            molecule.addh()
        if self.config.make_3d and not molecule.OBMol.HasNonZeroCoords():
            molecule.make3D()
        molecule.calccharges(model=self.config.charge_model)
        return molecule

    def save_molecule(self, molecule: pybel.Molecule, outpath: Union[str, os.PathLike], 
                      out_format: str = "pdbqt", overwrite: bool = False) -> None:
        with pybel.Outputfile(format=out_format, filename=str(outpath), overwrite=overwrite) as out:
            out.write(molecule)

class Preprocessor:
    def __init__(self, config: MoleculePreparationConfig = MoleculePreparationConfig()):
        self.preparator = MoleculePreparator(config)

    def prepare_receptor(self, receptor_path: Union[str, os.PathLike], 
                         output_path: Union[str, os.PathLike]) -> None:
        molecules = MoleculeReader.read_molecule(receptor_path)
        prepared_receptor = self.preparator.prepare_molecule(molecules[0])
        self.preparator.save_molecule(prepared_receptor, output_path)

    def prepare_ligand(self, ligand_path: Union[str, os.PathLike], 
                       output_path: Union[str, os.PathLike], 
                       in_format: str = "pdb") -> None:
        molecules = MoleculeReader.read_molecule(ligand_path, in_format)
        prepared_ligand = self.preparator.prepare_molecule(molecules[0])
        self.preparator.save_molecule(prepared_ligand, output_path)

class Docking:
    def __init__(self, receptor_path: str, box: Box, num_poses: int = 5, exhaustiveness: int = 8):
        self.receptor_path = receptor_path
        self.box = box
        self.num_poses = num_poses
        self.exhaustiveness = exhaustiveness

    def dock_one(self, ligand_path: str, out_path: str) -> str:
        import subprocess
        cmd = [
            "smina",
            "--receptor", self.receptor_path,
            "--ligand", ligand_path,
            "--out", out_path,
            "--center_x", str(self.box.center.x),
            "--center_y", str(self.box.center.y),
            "--center_z", str(self.box.center.z),
            "--size_x", str(self.box.size.x),
            "--size_y", str(self.box.size.y),
            "--size_z", str(self.box.size.z),
            "--num_modes", str(self.num_poses),
            "--exhaustiveness", str(self.exhaustiveness)
        ]
        return subprocess.check_output(cmd, universal_newlines=True)

    def parse_output(self, output_text: str) -> pd.DataFrame:
        import pandas as pd
        smina_delimiter = "-----+------------+----------+----------"
        lines = output_text.split("\n")
        rows_index = lines.index(smina_delimiter)
        results = [list(map(float, line.split()[1:])) for line in lines[rows_index + 1 : -3]]
        return pd.DataFrame(results, columns=["affinity", "rmsd_lb", "rmsd_ub"])

def create_box_from_ligand(ligand: mdtraj.Trajectory) -> Box:
    xyz = ligand.xyz[0] * 10  # convert to Angstrom from nm
    pocket_center = (xyz.max(axis=0) + xyz.min(axis=0)) / 2
    pocket_size = xyz.max(axis=0) - xyz.min(axis=0) + 5
    return Box.from_array(pocket_center, pocket_size)

def visualize_docking(receptor_path: str, ligand_path: str) -> py3Dmol.view:
    view = py3Dmol.view()
    view.addModel(open(receptor_path, 'r').read())
    view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
    view.addModel(open(ligand_path, 'r').read())
    view.setStyle({'model': -1}, {"stick" :  {'color': "yellow"}})
    view.zoomTo()
    return view

In [ ]:
# Prepare molecules
prep = Preprocessor()
prep.prepare_receptor("data/CH08_6vhn_prepared.pdb", "artifacts/ch08/dock_inputs/receptor.pdbqt")
prep.prepare_ligand("data/CH08_ligand.pdb", "artifacts/ch08/dock_inputs/ligand.pdbqt", in_format="pdb")

*** Open Babel Warning  in parseConectRecord
  Problems reading a CONECT record.
  According to the PDB specification,
  columns 7-11 should contain the serial number of an atom.
  No atom was found with this serial number.
  THIS CONECT RECORD WILL BE IGNORED.

*** Open Babel Warning  in parseConectRecord
  Problems reading a CONECT record.
  According to the PDB specification,
  columns 7-11 should contain the serial number of an atom.
  No atom was found with this serial number.
  THIS CONECT RECORD WILL BE IGNORED.

*** Open Babel Warning  in parseConectRecord
  Problems reading a CONECT record.
  According to the PDB specification,
  columns 7-11 should contain the serial number of an atom.
  No atom was found with this serial number.
  THIS CONECT RECORD WILL BE IGNORED.

*** Open Babel Warning  in parseConectRecord
  Problems reading a CONECT record.
  According to the PDB specification,
  columns 7-11 should contain the serial number of an atom.
  No atom was found with this se

In [ ]:
# Create docking box
ligand = mdtraj.load("data/CH08_ligand.pdb")
box = create_box_from_ligand(ligand)

In [ ]:
%conda install conda-forge::smina --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 24.9.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.9.1



## Package Plan ##

  environment location: /home/nflynn/anaconda3/envs/ml-drug-discovery

  added / updated specs:
    - conda-forge::smina


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |       2_kmp_llvm           6 KB  conda-forge
    ca-certificates-2024.9.24  |       h06a4308_0         130 KB
    cairo-1.18.0               |       hbb29018_2         961 KB  conda-forge
    expat-2.6.3                |       h6a678d5_0         176 KB
    font-ttf-dejavu-sans-mono-2.37|       hd3eb1b0_0         335 KB
    font-ttf-inconsolata-2.001 |       hcb226

In [ ]:
# Perform docking
docker = Docking("artifacts/ch08/dock_inputs/receptor.pdbqt", box)
docking_output = docker.dock_one("artifacts/ch08/dock_inputs/ligand.pdbqt", "artifacts/ch08/dock_outputs/ligand_out.sdf")
results = docker.parse_output(docking_output)
print(results)

*** Open Babel Error  in openLib
  /home/nflynn/anaconda3/envs/ml-drug-discovery/lib/python3.10/site-packages/openbabel/lib/openbabel/3.1.0/APIInterface.so did not load properly.
 Error: /home/nflynn/anaconda3/envs/ml-drug-discovery/lib/python3.10/site-packages/openbabel/lib/openbabel/3.1.0/APIInterface.so: undefined symbol: _ZN9OpenBabel8OBFormat7DisplayERSsPKcS3_
*** Open Babel Error  in openLib
  /home/nflynn/anaconda3/envs/ml-drug-discovery/lib/python3.10/site-packages/openbabel/lib/openbabel/3.1.0/CSRformat.so did not load properly.
 Error: /home/nflynn/anaconda3/envs/ml-drug-discovery/lib/python3.10/site-packages/openbabel/lib/openbabel/3.1.0/CSRformat.so: undefined symbol: _ZN9OpenBabel8OBFormat7DisplayERSsPKcS3_
*** Open Babel Error  in openLib
  /home/nflynn/anaconda3/envs/ml-drug-discovery/lib/python3.10/site-packages/openbabel/lib/openbabel/3.1.0/MCDLformat.so did not load properly.
 Error: /home/nflynn/anaconda3/envs/ml-drug-discovery/lib/python3.10/site-packages/openbabel/

CalledProcessError: Command '['smina', '--receptor', 'artifacts/ch08/dock_inputs/receptor.pdbqt', '--ligand', 'artifacts/ch08/dock_inputs/ligand.pdbqt', '--out', 'artifacts/ch08/dock_outputs/ligand_out.sdf', '--center_x', '-51.568', '--center_y', '0.7765', '--center_z', '23.5065', '--size_x', '11.748001', '--size_y', '12.989', '--size_z', '17.699', '--num_modes', '5', '--exhaustiveness', '8']' returned non-zero exit status 255.

In [ ]:
# Visualize docking results
view = visualize_docking('data/CH08_6vhn_prepared.pdb', 'artifacts/ch08/dock_outputs/ligand_out.sdf')
view.show()

In [ ]:






    # Read and process docking poses
    poses = dm.read_sdf("outputs/ligand_out.sdf", as_df=True, mol_column="mols", n_jobs=-1)

# Scratch

In [ ]:
def load_sdf(file_path):
    molecules = list(Chem.SDMolSupplier(file_path))
    return [mol for mol in molecules if mol is not None]

temp = load_sdf('data/Enamine_Hinge_Binders_Library_plated_24000cmds_20210316.sdf')

In [ ]:
temp[:5]

In [ ]:
suppl = Chem.SDMolSupplier('data/Enamine_Hinge_Binders_Library_plated_24000cmds_20210316.sdf')
idx = 0
for mol in suppl:
  if idx > 5: break
  print(mol.GetNumAtoms())
  idx += 1


12
12
13
13
13
14
